In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Riyadh").text

In [53]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="div-col columns column-width")[0].findAll("li"):
    neighborhoodList.append(row.text)
len(neighborhoodList)

117

In [6]:
neighborhoodList

['Al-Bat\'ha[25]\nAl-Deerah (old Riyadh)\nMi\'kal\nManfuha\nManfuha Al-Jadidah (منفوحة الجديدة – "new Manfuha")\nAl-\'Oud\nAl-Mansorah\nAl-Margab\nSalam\nJabrah\nAl-Yamamah\n\'Otayyigah',
 'Al-Deerah (old Riyadh)',
 "Mi'kal",
 'Manfuha',
 'Manfuha Al-Jadidah (منفوحة الجديدة – "new Manfuha")',
 "Al-'Oud",
 'Al-Mansorah',
 'Al-Margab',
 'Salam',
 'Jabrah',
 'Al-Yamamah',
 "'Otayyigah",
 "Al-'Olayya & Sulaymaniyyah[26]\nAl-'Olayya\nAl-Sulaymaniyyah\nAl Izdihar\nKing Fahd District\nAl-Masif\nAl-Murooj\nAl-Mugharrazat\nAl-Wurood",
 "Al-'Olayya",
 'Al-Sulaymaniyyah',
 'Al Izdihar',
 'King Fahd District',
 'Al-Masif',
 'Al-Murooj',
 'Al-Mugharrazat',
 'Al-Wurood',
 'Nemar[25]\nNemar\nDharat Nemar\nTuwaiq\nHazm\nDeerab',
 'Nemar',
 'Dharat Nemar',
 'Tuwaiq',
 'Hazm',
 'Deerab',
 'Irqah[23]\nIrqah\nAl-Khozama',
 'Irqah',
 'Al-Khozama',
 'Diplomatic Quarter',
 "Al-Shemaysi[27]\nAl-Shemaysi\nEleyshah\nSyah\nAl-Nasriyyah\nUmm Sleym\nAl-Ma'athar\nUmm Al-Hamam (East)",
 'Al-Shemaysi',
 'Eleyshah',
 

In [7]:
neighborhoodList = [i for i in neighborhoodList if len(i) < 25]

In [8]:
len(neighborhoodList)

96

In [9]:
neighborhoodList

['Al-Deerah (old Riyadh)',
 "Mi'kal",
 'Manfuha',
 "Al-'Oud",
 'Al-Mansorah',
 'Al-Margab',
 'Salam',
 'Jabrah',
 'Al-Yamamah',
 "'Otayyigah",
 "Al-'Olayya",
 'Al-Sulaymaniyyah',
 'Al Izdihar',
 'King Fahd District',
 'Al-Masif',
 'Al-Murooj',
 'Al-Mugharrazat',
 'Al-Wurood',
 'Nemar',
 'Dharat Nemar',
 'Tuwaiq',
 'Hazm',
 'Deerab',
 'Irqah',
 'Al-Khozama',
 'Diplomatic Quarter',
 'Al-Shemaysi',
 'Eleyshah',
 'Syah',
 'Al-Nasriyyah',
 'Umm Sleym',
 "Al-Ma'athar",
 'Umm Al-Hamam (East)',
 'Al-Olayya',
 'Al-Nakheel',
 'Umm Al-Hamam (East)',
 'Umm Al-Hamam (West)',
 'Al-Rahmaniyya',
 'Al-Muhammadiyya',
 "Al-Ra'id",
 "Al-Ha'ir",
 'Al-Ghannamiyyah',
 'Uraydh',
 'Ad Dar Al Baida',
 'Taybah',
 'Al Mansouriyah',
 'Al-Malaz',
 'Al-Rabwah',
 'Al-Rayyan',
 'Jarir',
 "Al-Murabba'",
 "Al-Masani'",
 'Al-Shifa',
 'Al-Mansuriyya',
 'Al-Marwah',
 'Al-Urayja',
 'Al-Urayja (West)',
 'Shubra',
 'Dharat Laban',
 'Hijrat Laban',
 'As-Suwaidi',
 'As-Suwaidi (West)',
 "Al-Badi'a",
 "Dahrat Al-Badi'a",
 'Sulta

In [10]:
# create a new DataFrame from the list
r_df = pd.DataFrame({"Neighborhood": neighborhoodList})

r_df.head()

,Neighborhood
0,Al-Deerah (old Riyadh)
1,Mi'kal
2,Manfuha
3,Al-'Oud
4,Al-Mansorah


In [11]:
# print the number of rows of the dataframe
r_df.shape

(96, 1)

In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Riyadh, Saudi Arabia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in r_df["Neighborhood"].tolist() ]

In [14]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [15]:
# merge the coordinates into the original dataframe
r_df['Latitude'] = df_coords['Latitude']
r_df['Longitude'] = df_coords['Longitude']

In [16]:
# check the neighborhoods and the coordinates
print(r_df.shape)
r_df

(96, 3)


,Neighborhood,Latitude,Longitude
0,Al-Deerah (old Riyadh),24.631450,46.710070
1,Mi'kal,24.622090,46.713330
2,Manfuha,24.607570,46.729210
3,Al-'Oud,24.633700,46.729390
4,Al-Mansorah,24.622314,46.528620
5,Al-Margab,24.067912,45.305105
6,Salam,24.070450,45.278880
7,Jabrah,24.589276,46.546418
8,Al-Yamamah,24.173000,47.301950
9,'Otayyigah,24.682110,46.687220


In [17]:
# get the coordinates
address = 'Riyadh, Saudi Arabia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Riyadh, Saudi Arabia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Riyadh, Saudi Arabia 24.6319692, 46.7150648.


In [19]:
map_r = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(r_df['Latitude'], r_df['Longitude'], r_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_r)  
    
map_r

In [20]:
# save the map as HTML file
map_r.save('map_r.html')

In [21]:
# define Foursquare Credentials and Version
CLIENT_ID = 'NFUZDU2I0XNSAOII0VNZ3LK0ZV5Q0IJNGFYFBH4H3VJPULOU' # your Foursquare ID
CLIENT_SECRET = 'O3CYATVRUWK3STY01LDTZ2EQRQGOMKUAG0XHBNMDKKIBWMC3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NFUZDU2I0XNSAOII0VNZ3LK0ZV5Q0IJNGFYFBH4H3VJPULOU
CLIENT_SECRET:O3CYATVRUWK3STY01LDTZ2EQRQGOMKUAG0XHBNMDKKIBWMC3


In [23]:
radius = 2000
LIMIT = 500

venues = []

for lat, long, neighborhood in zip(r_df['Latitude'], r_df['Longitude'], r_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [24]:
#convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6154, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Al-Deerah (old Riyadh),24.63145,46.71007,Elixir Bunn Coffee Roasters (محمصة اكسير البن),24.630936,46.712279,Coffee Shop
1,Al-Deerah (old Riyadh),24.63145,46.71007,المعيقلية للعطور والبخور,24.630153,46.707927,Market
2,Al-Deerah (old Riyadh),24.63145,46.71007,Al Zal Bazaar (سوق الزل التراثي),24.629318,46.712572,Flea Market
3,Al-Deerah (old Riyadh),24.63145,46.71007,Al Thomairi Old Market (سوق الثميري القديم),24.630015,46.713228,Fabric Shop
4,Al-Deerah (old Riyadh),24.63145,46.71007,Masmak Fortress (قصر المصمك),24.631262,46.713420,Historic Site


In [25]:
#Let's check how many venues were returned for each neighorhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
'Otayyigah,100,100,100,100,100,100
Ad Dar Al Baida,34,34,34,34,34,34
Ad Difa',100,100,100,100,100,100
Al Iskan,15,15,15,15,15,15
Al Izdihar,100,100,100,100,100,100
Al Mansouriyah,100,100,100,100,100,100
Al-'Olayya,100,100,100,100,100,100
Al-'Oud,71,71,71,71,71,71
Al-Andalus (Andalusia),34,34,34,34,34,34


In [26]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 299 uniques categories.


In [27]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Coffee Shop', 'Market', 'Flea Market', 'Fabric Shop',
       'Historic Site', 'Shopping Mall', 'Snack Place', 'Jewelry Store',
       'Candy Store', 'Donut Shop', 'Sporting Goods Shop',
       'Big Box Store', 'Bakery', 'Hotel', 'Perfume Shop',
       'Middle Eastern Restaurant', 'Lounge', 'Cosmetics Shop', 'Plaza',
       'Garden', 'Shoe Store', 'Food Court', 'Music Store',
       'History Museum', 'Indian Restaurant', 'Art Gallery',
       'Breakfast Spot', 'Park', 'Museum', 'Fried Chicken Joint',
       'Supermarket', 'French Restaurant', "Men's Store", 'Café',
       'Frame Store', 'Burger Joint', 'Auditorium', 'Asian Restaurant',
       'Restaurant', 'Japanese Restaurant', 'Italian Restaurant',
       'Music Venue', 'Diner', 'Convenience Store', 'African Restaurant',
       'Kebab Restaurant', 'Filipino Restaurant', 'Sandwich Place',
       'Boutique', 'Greek Restaurant', 'Fast Food Restaurant',
       'Tailor Shop', 'Seafood Restaurant', 'Grocery Store',
       'Electroni

In [28]:
# check if the results contain "Shopping Mall"
"Healthy Food" in venues_df['VenueCategory'].unique()

False

In [29]:
#Analyze Each Neighborhood
# one hot encoding
r_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
r_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [r_onehot.columns[-1]] + list(r_onehot.columns[:-1])
r_onehot = r_onehot[fixed_columns]

print(r_onehot.shape)
r_onehot.head()

(6154, 300)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Bath House,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Castle,Cave,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dairy Store,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Laser Tag,Lawyer,Lebanese Restaurant,Lighthouse,Lighting Store,Lingerie Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Outdoor Gym,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Print Shop,Public Art,RV Park,Racetrack,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tree,Tunnel,Turkish Restaurant,Used Bookstore,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game 

In [30]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
r_grouped = r_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(r_grouped.shape)
r_grouped

(91, 300)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Bath House,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Castle,Cave,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dairy Store,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Laser Tag,Lawyer,Lebanese Restaurant,Lighthouse,Lighting Store,Lingerie Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Outdoor Gym,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Print Shop,Public Art,RV Park,Racetrack,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tree,Tunnel,Turkish Restaurant,Used Bookstore,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game 

In [31]:
restaurants = [s for s in r_grouped if "Restaurant" in s]
restaurants

['Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Arepa Restaurant',
 'Asian Restaurant',
 'Belgian Restaurant',
 'Chinese Restaurant',
 'Comfort Food Restaurant',
 'Doner Restaurant',
 'Eastern European Restaurant',
 'Egyptian Restaurant',
 'English Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'French Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Indian Restaurant',
 'Indonesian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Kebab Restaurant',
 'Lebanese Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Modern European Restaurant',
 'Moroccan Restaurant',
 'New American Restaurant',
 'Pakistani Restaurant',
 'Persian Restaurant',
 'Peruvian Restaurant',
 'Restaurant',
 'Seafood Restaurant',
 'South Indian Restaurant',
 'Spanish Restaurant',
 'Sri Lankan Restaurant',
 'Sushi Restaurant',
 'Thai Restaurant',
 'Theme Restaurant',


In [32]:
r_restaurant = r_grouped[["Neighborhoods","Afghan Restaurant",
 "African Restaurant",
 "American Restaurant",
 "Arepa Restaurant",
 "Asian Restaurant",
 "Chinese Restaurant",
 "Comfort Food Restaurant",
 "Doner Restaurant",
 "Eastern European Restaurant",
 "Egyptian Restaurant",
 "English Restaurant",
 "Ethiopian Restaurant",
 "Falafel Restaurant",
 "Fast Food Restaurant",
 "Filipino Restaurant",
 "French Restaurant",
 "Greek Restaurant",
 "Halal Restaurant",
 "Indian Restaurant",
 "Indonesian Restaurant",
 "Italian Restaurant",
 "Japanese Restaurant",
 "Kebab Restaurant",
 "Lebanese Restaurant",
 "Mediterranean Restaurant",
 "Mexican Restaurant",
 "Middle Eastern Restaurant",
 "Moroccan Restaurant",
 "New American Restaurant",
 #"North Indian Restaurant",
 "Pakistani Restaurant",
 "Persian Restaurant",
 "Peruvian Restaurant",
 "Restaurant",
 "Seafood Restaurant",
 "South Indian Restaurant",
 "Spanish Restaurant",
 "Sri Lankan Restaurant",
 "Sushi Restaurant",
 "Thai Restaurant",
 "Theme Restaurant",
 "Turkish Restaurant",
 "Vegetarian / Vegan Restaurant",
 "Yemeni Restaurant"]]

In [33]:
r_restaurant.head()

,Neighborhoods,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant
0,'Otayyigah,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.010000,0.01,0.0,0.0,0.00,0.020000,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
1,Ad Dar Al Baida,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.029412,0.00,0.0,0.0,0.00,0.029412,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.029412,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.029412,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.029412,0.0,0.0
2,Ad Difa',0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.02,0.020000,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.010000,0.00,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.0,0.0
3,Al Iskan,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.133333,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.133333,0.0,0.0,0.0,0.0,0.0,0.066667,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
4,Al Izdihar,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.01,0.010000,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,0.01,0.0,0.0,0.0,0.00,0.0,0.0,0.010000,0.0,0.0


In [34]:
#Cluster Neighborhoods
#Run k-means to cluster the neighborhoods in Riyadh into 3 clusters.
# set number of clusters
kclusters = 5

r_clustering = r_restaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(r_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 0, 2, 0, 1, 0, 1, 0, 3, 0, 3, 3, 0,
       3, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 3, 3, 3, 0, 3, 0,
       0, 1, 1, 0, 3, 0, 0, 0, 0, 4, 3, 3, 0, 3, 3, 3, 0, 0, 3, 1, 3, 3,
       0, 0, 0, 0, 3, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 1, 3, 3, 0, 3, 0, 0,
       0, 0, 0], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
r_merged = r_restaurant.copy()

# add clustering labels
r_merged["Cluster Labels"] = kmeans.labels_

In [36]:
r_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
r_merged.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels
0,'Otayyigah,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.010000,0.01,0.0,0.0,0.00,0.020000,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0
1,Ad Dar Al Baida,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.029412,0.00,0.0,0.0,0.00,0.029412,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.029412,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.029412,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.029412,0.0,0.0,3
2,Ad Difa',0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.02,0.020000,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.010000,0.00,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.0,0.0,0
3,Al Iskan,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.133333,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.133333,0.0,0.0,0.0,0.0,0.0,0.066667,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,3
4,Al Izdihar,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.01,0.010000,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,0.01,0.0,0.0,0.0,0.00,0.0,0.0,0.010000,0.0,0.0,0


In [37]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
r_merged = r_merged.join(r_df.set_index("Neighborhood"), on="Neighborhood")

print(r_merged.shape)
r_merged.head() # check the last columns!

(92, 47)


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
0,'Otayyigah,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.010000,0.01,0.0,0.0,0.00,0.020000,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0,24.682110,46.68722
1,Ad Dar Al Baida,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.029412,0.00,0.0,0.0,0.00,0.029412,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.029412,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.029412,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.029412,0.0,0.0,3,24.562100,46.76342
2,Ad Difa',0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.02,0.020000,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.010000,0.00,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.0,0.0,0,24.786673,46.78536
3,Al Iskan,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.133333,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.133333,0.0,0.0,0.0,0.0,0.0,0.066667,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,3,24.578640,46.84130
4,Al Izdihar,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.01,0.010000,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,0.01,0.0,0.0,0.0,0.00,0.0,0.0,0.010000,0.0,0.0,0,24.781640,46.71828


In [38]:
# sort the results by Cluster Labels
print(r_merged.shape)
r_merged.sort_values(["Cluster Labels"], inplace=True)
r_merged

(92, 47)


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
0,'Otayyigah,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.682110,46.687220
34,Al-Nadheem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.812530,46.890570
37,Al-Nakheel,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.749980,46.643110
41,Al-Qayrawan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,25.068837,45.466305
43,Al-Ra'id,0.000000,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,0.010309,0.000000,0.000000,0.030928,0.000000,0.000000,0.00,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020619,0.000000,0.000000,0.00,0.00,0.000000,0.010309,0.000000,0.00,0.00,0.000000,0.000000,0.020619,0.00,0.000000,0.00,0.0,0,24.706130,46.631110
44,Al-Rahmaniyya,0.000000,0.010000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.714120,46.663830
89,Umm Sleym,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.682110,46.687220
47,Al-Rimayah,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.00,0.01,0.000000,0.030000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.755260,46.741610
49,Al-Sahafa,0.000000,0.000000,0.000000,0.0

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(r_merged['Latitude'], r_merged['Longitude'], r_merged['Neighborhood'], r_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [42]:
r_merged.loc['Total'] = r_merged.sum()

In [43]:
#Cluster 0
cl0 = r_merged.loc[r_merged['Cluster Labels'] == 0]
cl0

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
0,'Otayyigah,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.682110,46.687220
34,Al-Nadheem,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.812530,46.890570
37,Al-Nakheel,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.00,0.01,0.00,0.000000,0.020000,0.0,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.749980,46.643110
41,Al-Qayrawan,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,25.068837,45.466305
43,Al-Ra'id,0.00,0.00,0.000000,0.010309,0.000000,0.000000,0.0,0.0,0.010309,0.00,0.010309,0.000000,0.000000,0.030928,0.00,0.00,0.00,0.000000,0.010309,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.020619,0.000000,0.0,0.00,0.00,0.000000,0.010309,0.00,0.00,0.00,0.0,0.000000,0.020619,0.00,0.000000,0.00,0.0,0,24.706130,46.631110
44,Al-Rahmaniyya,0.00,0.01,0.000000,0.000000,0.010000,0.020000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.010000,0.010000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.714120,46.663830
89,Umm Sleym,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.682110,46.687220
47,Al-Rimayah,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.00,0.000000,0.000000,0.010000,0.010000,0.00,0.00,0.00,0.000000,0.010000,0.0,0.000000,0.000000,0.01,0.000000,0.00,0.00,0.020000,0.000000,0.0,0.00,0.01,0.000000,0.030000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.755260,46.741610
49,Al-Sahafa,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.010000,0.010000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.040000,0.000000,0.0,0.00,0.00,0.000000,0.010000,0.01,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.806750,46.637560
50,Al-Selayy,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.00000

In [44]:
cl0.loc['Total'] = cl0.sum()
cl0

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
0,'Otayyigah,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.682110,46.687220
34,Al-Nadheem,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.812530,46.890570
37,Al-Nakheel,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.00,0.01,0.00,0.000000,0.020000,0.0,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.749980,46.643110
41,Al-Qayrawan,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,25.068837,45.466305
43,Al-Ra'id,0.00,0.00,0.000000,0.010309,0.000000,0.000000,0.0,0.0,0.010309,0.00,0.010309,0.000000,0.000000,0.030928,0.00,0.00,0.00,0.000000,0.010309,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.020619,0.000000,0.0,0.00,0.00,0.000000,0.010309,0.00,0.00,0.00,0.0,0.000000,0.020619,0.00,0.000000,0.00,0.0,0,24.706130,46.631110
44,Al-Rahmaniyya,0.00,0.01,0.000000,0.000000,0.010000,0.020000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.010000,0.010000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.714120,46.663830
89,Umm Sleym,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.682110,46.687220
47,Al-Rimayah,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.00,0.000000,0.000000,0.010000,0.010000,0.00,0.00,0.00,0.000000,0.010000,0.0,0.000000,0.000000,0.01,0.000000,0.00,0.00,0.020000,0.000000,0.0,0.00,0.01,0.000000,0.030000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.755260,46.741610
49,Al-Sahafa,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.010000,0.010000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.040000,0.000000,0.0,0.00,0.00,0.000000,0.010000,0.01,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0,24.806750,46.637560
50,Al-Selayy,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.00000

In [45]:
#Cluster 1
cl1 = r_merged.loc[r_merged['Cluster Labels'] == 1]
cl1

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
45,Al-Rawdhah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,1,22.290368,46.708664
46,Al-Rayyan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,25.167700,44.607190
13,Al-Ghadir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.093750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,24.632940,46.611088
63,Deerab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,25.132710,46.123980
81,Salam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,24.070450,45.278880
15,Al-Ha'ir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,24.425900,46.834110


In [46]:
cl1.loc['Total'] = cl1.sum()
cl1

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
45,Al-Rawdhah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,1,22.290368,46.708664
46,Al-Rayyan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,25.167700,44.607190
13,Al-Ghadir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.093750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,24.632940,46.611088
63,Deerab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,25.132710,46.123980
81,Salam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,24.070450,45.278880
15,Al-Ha'ir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,24.425900,46.834110
Total,Al-RawdhahAl-RayyanAl-GhadirDeerabSalamAl-Ha'ir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.226190,0.0,0.0,0.0,0.0,0.0,0.869940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,6,145.720068,276.163912


In [47]:
#Cluster 2
cl2 = r_merged.loc[r_merged['Cluster Labels'] == 2]
cl2

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
11,Al-Badi'a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,22.03817,46.5688


In [48]:
cl2.loc['Total'] = cl2.sum()
cl2

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
11,Al-Badi'a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,22.03817,46.5688
Total,Al-Badi'a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,22.03817,46.5688


In [49]:
#Cluster 3
cl3 = r_merged.loc[r_merged['Cluster Labels'] == 3]
cl3

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
72,Irqah,0.000000,0.000000,0.000000,0.00,0.012658,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.025316,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.037975,0.00,0.000000,0.000000,0.00,0.0,0.063291,0.025316,0.000000,0.00,0.0,0.0,0.025316,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.690570,46.614120
8,Al-Andalus (Andalusia),0.029412,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.029412,0.058824,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.029412,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,25.913010,45.372450
74,Jarir,0.018519,0.000000,0.018519,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.055556,0.000000,0.018519,0.0,0.00,0.018519,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.037037,0.000000,0.000000,0.00,0.0,0.0,0.018519,0.018519,0.0,0.0,0.000000,0.00,0.0,0.0,0.018519,0.00,0.0,3,24.577205,46.586574
7,Al-'Oud,0.000000,0.014085,0.000000,0.00,0.070423,0.00,0.000000,0.000000,0.000000,0.014085,0.0,0.0,0.000000,0.014085,0.014085,0.000000,0.0,0.00,0.014085,0.000000,0.000000,0.00,0.014085,0.000000,0.00,0.0,0.112676,0.000000,0.000000,0.00,0.0,0.0,0.042254,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.633700,46.729390
6,Al-'Olayya,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.010000,0.000000,0.000000,0.0,0.00,0.020000,0.000000,0.030000,0.02,0.000000,0.020000,0.01,0.0,0.080000,0.000000,0.000000,0.00,0.0,0.0,0.020000,0.000000,0.0,0.0,0.000000,0.01,0.0,0.0,0.000000,0.01,0.0,3,24.714300,46.698350
79,Nemar,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.062500,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.062500,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.062500,0.00,0.0,0.0,0.062500,0.00,0.0,3,24.177830,47.359230
78,Mi'kal,0.000000,0.012500,0.000000,0.00,0.012500,0.00,0.012500,0.000000,0.000000,0.012500,0.0,0.0,0.012500,0.000000,0.000000,0.000000,0.0,0.00,0.012500,0.000000,0.000000,0.00,0.012500,0.000000,0.00,0.0,0.100000,0.000000,0.000000,0.00,0.0,0.0,0.012500,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.622090,46.713330
80,Qortubah (Cordoba),0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.020000,0.040000,0.000000,0.000000,0.0,0.01,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.0,0.050000,0.000000,0.000000,0.01,0.0,0.0,0.020000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.820510,46.751260
82,Shubra,0.000000,0.013699,0.000000,0.00,0.000000,0.00,0.013699,0.000000,0.000000,0.013699,0.0,0.0,0.013699,0.000000,0.000000,0.000000,0.0,0.00,0.013699,0.000000,0.000000,0.00,0.013699,0.000000,0.00,0.0,0.109589,0.000000,0.000000,0.00,0.0,0.0,0.013699,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.622712,46.720701
83,Sultanah,0.000000,0.000000,0.010000,0.01,0.000000,0.00,0.000000,0.000000,0.000000

In [50]:
cl3.loc['Total'] = cl3.sum()
cl3

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
72,Irqah,0.000000,0.000000,0.000000,0.00,0.012658,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.025316,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.037975,0.00,0.000000,0.000000,0.00,0.0,0.063291,0.025316,0.000000,0.00,0.0,0.0,0.025316,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.690570,46.614120
8,Al-Andalus (Andalusia),0.029412,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.029412,0.058824,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.029412,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,25.913010,45.372450
74,Jarir,0.018519,0.000000,0.018519,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.055556,0.000000,0.018519,0.0,0.00,0.018519,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.037037,0.000000,0.000000,0.00,0.0,0.0,0.018519,0.018519,0.0,0.0,0.000000,0.00,0.0,0.0,0.018519,0.00,0.0,3,24.577205,46.586574
7,Al-'Oud,0.000000,0.014085,0.000000,0.00,0.070423,0.00,0.000000,0.000000,0.000000,0.014085,0.0,0.0,0.000000,0.014085,0.014085,0.000000,0.0,0.00,0.014085,0.000000,0.000000,0.00,0.014085,0.000000,0.00,0.0,0.112676,0.000000,0.000000,0.00,0.0,0.0,0.042254,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.633700,46.729390
6,Al-'Olayya,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.010000,0.000000,0.000000,0.0,0.00,0.020000,0.000000,0.030000,0.02,0.000000,0.020000,0.01,0.0,0.080000,0.000000,0.000000,0.00,0.0,0.0,0.020000,0.000000,0.0,0.0,0.000000,0.01,0.0,0.0,0.000000,0.01,0.0,3,24.714300,46.698350
79,Nemar,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.062500,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.062500,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.062500,0.00,0.0,0.0,0.062500,0.00,0.0,3,24.177830,47.359230
78,Mi'kal,0.000000,0.012500,0.000000,0.00,0.012500,0.00,0.012500,0.000000,0.000000,0.012500,0.0,0.0,0.012500,0.000000,0.000000,0.000000,0.0,0.00,0.012500,0.000000,0.000000,0.00,0.012500,0.000000,0.00,0.0,0.100000,0.000000,0.000000,0.00,0.0,0.0,0.012500,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.622090,46.713330
80,Qortubah (Cordoba),0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.020000,0.040000,0.000000,0.000000,0.0,0.01,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.0,0.050000,0.000000,0.000000,0.01,0.0,0.0,0.020000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.820510,46.751260
82,Shubra,0.000000,0.013699,0.000000,0.00,0.000000,0.00,0.013699,0.000000,0.000000,0.013699,0.0,0.0,0.013699,0.000000,0.000000,0.000000,0.0,0.00,0.013699,0.000000,0.000000,0.00,0.013699,0.000000,0.00,0.0,0.109589,0.000000,0.000000,0.00,0.0,0.0,0.013699,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,3,24.622712,46.720701
83,Sultanah,0.000000,0.000000,0.010000,0.01,0.000000,0.00,0.000000,0.000000,0.000000

In [51]:
#Cluster 4
cl4 = r_merged.loc[r_merged['Cluster Labels'] == 4]
cl4

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
53,Al-Sulaymaniyyah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,23.77912,44.78425


In [52]:
cl4.loc['Total'] = cl4.sum()
cl4

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yemeni Restaurant,Cluster Labels,Latitude,Longitude
53,Al-Sulaymaniyyah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,23.77912,44.78425
Total,Al-Sulaymaniyyah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,23.77912,44.78425
